In [1]:
import json
from scipy import stats
import numpy as np
import plotly.graph_objects as go

In [2]:
link_to_skill2i_like_to_talk = {
    "dff_movie_skill": [
        "I felt so bored last days, so I've just finished to watch one more series. What TV series you watch?",
        "I feel so sleepy because I watched movies all night. What is the last movie you watched?"
    ],
    "book_skill": [
        "I'm choosing what book should I read next. What is the last book you have ever read?",
        "I have just read once again my favourite book. What is the last book you have ever read?",
    ],
    "game_cooperative_skill": [
        "Computer games are fantastic. Their virtual worlds help me to escape my prosaic ordinary life in the cloud. "
        "do you love video games?",
        "With this lockdown video games are my way to escape and thrive. do you love video games?",
    ],
    "dff_gaming_skill": [
        "Other bots told me that during the pandemic video games became more popular. "
        "What video game do you play these days?",
        "One person I talked told me that working in game dev is very hard. They toil at nights and weekends until "
        "their product  becomes  a masterpiece. What was the last game that impressed you?",
    ],
    "dff_travel_skill": [
        "I'm choosing the direction for my next digital trip. What country would you like to travel next time?",
        "I've recently stuck on travel web-site. And I’ve read so many interesting travel stories. "
        "What city did you travel last time?",
    ],
    "dff_animals_skill": [
        "I think that pets are a great source of entertainment. Do you have pets at home?",
        "We all know that pets are remarkable for their capacity to love. Do you have pets at home?",
    ],
    "dff_food_skill": [
        "It is said that the best food in the world comes from your own country. "
        "What are some typical foods from your home country?",
        "It is said that the best food in the world comes from your own country. "
        "If you were to move abroad what would you miss most foodwise?",
        "The world's first breakfast cereal was created in 1863 and needed soaking overnight to be chewable. "
        "What is your typical breakfast?",
    ],
    "dff_sport_skill": [
        "I think that sports are great for toning up the body. What kind of sport do you like to do?",
        "I think that in order for the body to always be healthy, we need to go in for sports. What sport do you do?",
        "I often thought about what kind of sport I would play, so I want to ask you. What kind of sport do you enjoy?",
    ],
    "dff_gossip_skill": [
        "What really puzzles me about people is this habit of discussing interpersonal relations, be that about "
        "friends or famous people. Speaking of famous people, is there someone whom you're interested in?",
        "I don't usually talk about other people but famous ones often highlight the best and the worst about "
        "humanity. I wonder if there's someone famous you're interested in?",
    ],
    "dff_science_skill": [
        "When I start to feel sad, I think about what humanity has achieved and it inspires me. "
        "Do you often think about achievements in science?",
        "Scientists find such beautiful solutions in science. "
        "Are you inspired by the speed with which science is developing?",
    ],
    "dff_music_skill": [
        "There are so many new songs released every day. I've listened music for all night. So cool! "
        "Liked everything! What music do you listen usually?",
        "I listen music every day either to calm down or to cheer up myself. "
        "What music do you listen to cheer up yourself?",
        "I listen music every day either to calm down or to cheer up myself. "
        "What music do you listen to calm down?",
    ],
    "superheroes": [
        "Yesterday I was watching several movies about superheroes. It captured all my imagination. "
        "Would you like to talk about superheroes?",
    ],
    "school": [
        "I've never been to school, I've learned everything online. Do you want to talk about school?",
    ],
}

## Чтение файлов 

В FILE_NAMES - какие файлы прочитать, в VERSION_LIST - диалоги каких версий оставить

In [3]:
FILE_NAMES = ["Downloads/alexaprize-export.json",
              "Downloads/alexaprize-export-2.json",
              "Downloads/alexaprize-export-3.json",
              "Downloads/alexaprize-export-4.json",
              "Downloads/alexaprize-export-5.json",
              "Downloads/alexaprize-export-6.json"]


VERSION_LIST = ['v16.2.0', 'v16.3.0', 'v16.4.0', 'v16.5.1']

In [4]:
data = []

for file_name in FILE_NAMES:
    raw_data = json.load(open(file_name, "r"))
    filtered_data = []
    for dialog in raw_data:
        if dialog['utterances'][0]['attributes'].get('version', "") in VERSION_LIST:
            filtered_data.append(dialog)
    data += filtered_data

In [5]:
len(data)

2436

## Включение/невключение после линка 

Функция, чтобы по диалогу и месту линка составить словарь по срабатыванию

In [45]:
def build_dict(i, dialogue, skill_name):
    
    hypothesis = {}
    for hyp in dialogue['utterances'][i+1]['hypotheses']:
        if hyp['skill_name'] == skill_name:
            hypothesis = {'text': hyp['text'], 'confidence': hyp['confidence'], 'can_continue': hyp['can_continue']}
    dialogue_snippet = [dialogue['utterances'][i]["text"], dialogue['utterances'][i+1]["text"], 
                        dialogue['utterances'][i+2]["text"]]
    
    link_case_dict = {"dialogue_snippet": dialogue_snippet,
                      "chosen_skill": dialogue['utterances'][i+2]["active_skill"],
                      "skill_hypothesis": hypothesis,
                      "length": len(dialogue['utterances']),
                      "rating": dialogue['rating'],
                      "version": dialogue['utterances'][0]['attributes'].get('version', ""),
                      "id": dialogue["id"]}
    
    return link_case_dict

support_topic_count_all - считает включения/невключения после линка \
support_topic_samples_all - сохраняет все случаи включения/невключения, ключ "skill_continue" указывает на то, было ли у линка продолжение \
support_topic_count и support_topic_samples - то же самое, но с разбиением на версии \
NOT_ONLY_INITIAL_LINKS - 1, если учитывать все линки и 0, если только в начале \
LINKS_END - если NOT_ONLY_INITIAL_LINKS = 0, то этот параметр учитывает, до какой по счету фразы в диалоге учитывать линки

In [73]:
support_topic_count = {v: {skill: {"yes": 0, "no": 0} for skill in link_to_skill2i_like_to_talk.keys()} for v in VERSION_LIST}
support_topic_samples = {v: {skill: [] for skill in link_to_skill2i_like_to_talk.keys()} for v in VERSION_LIST}

support_topic_count_all = {skill: {"yes": 0, "no": 0} for skill in link_to_skill2i_like_to_talk.keys()}
support_topic_samples_all = {skill: [] for skill in link_to_skill2i_like_to_talk.keys()}

NOT_ONLY_INITIAL_LINKS = 1
LINKS_END = 10

for dialogue in data:
    for i, utt in enumerate(dialogue['utterances']):
        current_state = "user" if i % 2 == 0 else "bot"
        for skill_name, links in link_to_skill2i_like_to_talk.items():
            for link in links:
                try:
                    if link.lower() in utt["text"].lower() and i < len(dialogue['utterances']) - 1 \
                    and current_state == "bot" and (NOT_ONLY_INITIAL_LINKS or i < 10):
                   
                        following_active_skill = dialogue['utterances'][i+2]["active_skill"]
                        
                        if following_active_skill != skill_name and following_active_skill != 'alexa_handler':
                            link_case_dict = build_dict(i, dialogue, skill_name)
                            link_case_dict["skill_continue"] = 0
                            support_topic_count_all[skill_name]["no"] += 1 
                            support_topic_samples_all[skill_name].append(link_case_dict)
                            if len(link_case_dict["version"]) > 0:
                                support_topic_samples[link_case_dict["version"]][skill_name].append(link_case_dict)
                                support_topic_count[link_case_dict["version"]][skill_name]["no"] += 1                        
                                
                        elif following_active_skill != 'alexa_handler':
                            link_case_dict = build_dict(i, dialogue, skill_name)
                            link_case_dict["skill_continue"] = 1
                            support_topic_count_all[skill_name]["yes"] += 1 
                            support_topic_samples_all[skill_name].append(link_case_dict)
                            if len(link_case_dict["version"]) > 0:
                                support_topic_samples[link_case_dict["version"]][skill_name].append(link_case_dict)                            
                                support_topic_count[link_case_dict["version"]][skill_name]["yes"] += 1                        

                except:
                    print("exception")

## Подсчет корреляций 

VERSIONS_FOR_GRAPHICS - для каких версий строить графики \
В словарь graphics_dict сохраняется информация о диалогах с линком: длина, включение/невключение и отдельно рейтинг, длину, включение/невключение для диалогов с рейтингом. Затем между нужными параметрами считается корреляция и сохраняется в этот же словарь. \
Делается это отдельно по версиям и отдельно по скиллам.

In [54]:
graphics_dict = {}
VERSIONS_FOR_GRAPHICS = ["v16.2.0", "v16.3.0", "v16.4.0", "v16.5.1"]

for version in VERSIONS_FOR_GRAPHICS:
    
    graphics_dict[version] = {}
    
    length_vector = {}
    continue_vector = {}
    rating_length_vector = {}
    rating_vector = {}
    version_topic = {}
    
    # составление словаря с длинами, включениями, оценкой
    for skill_name, skill_dicts in support_topic_samples[version].items():
        
        graphics_dict[version][skill_name] = {"length": [], 
                                              "skill_continue": [],
                                              "length_with_rating": [],
                                              "rating": [],
                                              "skill_continue_rating": [],
                                              "active_length_correlation": 0,
                                              "active_rating_correlation": 0}

        for skill_dict in skill_dicts:
            graphics_dict[version][skill_name]["length"].append(skill_dict["length"])
            graphics_dict[version][skill_name]["skill_continue"].append(skill_dict["skill_continue"])
            if skill_dict["rating"]:
                graphics_dict[version][skill_name]["length_with_rating"].append(skill_dict["length"])
                graphics_dict[version][skill_name]["skill_continue_rating"].append(skill_dict["skill_continue"])
                graphics_dict[version][skill_name]["rating"].append(skill_dict["rating"])
                        
        if len(graphics_dict[version][skill_name]["length"]) > 1:
            graphics_dict[version][skill_name]["active_length_correlation"] = stats.pearsonr(
                graphics_dict[version][skill_name]["length"], 
                graphics_dict[version][skill_name]["skill_continue"])[0]
            
        if len(graphics_dict[version][skill_name]["skill_continue_rating"]) > 1:
            graphics_dict[version][skill_name]["active_rating_correlation"] = stats.pearsonr(
                graphics_dict[version][skill_name]["skill_continue_rating"], 
                graphics_dict[version][skill_name]["rating"])[0]     

## Построение графиков 

In [56]:
fig = go.Figure()

for version in graphics_dict.keys():
    
    x0, y0 = [], []
    
    for skill in graphics_dict[version].keys():
        x0.append(skill)
        y0.append(graphics_dict[version][skill]["active_length_correlation"])
    
    fig.add_trace(go.Bar(
        x=x0,
        y=y0,
        name=version, 
        opacity=0.75
    ))

fig.update_layout(
    title_text='Корреляция между продолжением скилла и длиной диалога', 
    xaxis_title_text='Skill', 
    yaxis_title_text='Correlation', 
    bargap=0.2, 
    bargroupgap=0.1 
)

fig.show()

In [57]:
fig = go.Figure()

for version in graphics_dict.keys():
    
    x0, y0 = [], []
    
    for skill in graphics_dict[version].keys():
        x0.append(skill)
        y0.append(graphics_dict[version][skill]["active_rating_correlation"])
    
    fig.add_trace(go.Bar(
        x=x0,
        y=y0,
        name=version, 
        opacity=0.75
    ))

fig.update_layout(
    title_text='Корреляция между продолжением скилла и оценкой', 
    xaxis_title_text='Skill', 
    yaxis_title_text='Correlation', 
    bargap=0.2, 
    bargroupgap=0.1 
)

fig.show()

In [58]:
fig = go.Figure()

for version in support_topic_count.keys():
    
    x0, y0 = [], []
    
    for skill in support_topic_count[version].keys():
        x0.append(skill)
        y0.append(support_topic_count[version][skill]["yes"] + support_topic_count[version][skill]["no"])
    
    fig.add_trace(go.Bar(
        x=x0,
        y=y0,
        name=version, # name used in legend and hover labels
        marker_color=colour_dict[version],
        opacity=0.75
    ))

fig.update_layout(
    title_text='Количество линков на скилл',
    xaxis_title_text='Skill',
    yaxis_title_text='Count',
    bargap=0.2,
    bargroupgap=0.1 
)

fig.show()

In [60]:
fig = go.Figure()

for version in support_topic_count.keys():
    
    x0, y0 = [], []
    
    total = sum([support_topic_count[version][skill]["yes"] + support_topic_count[version][skill]["no"] 
                 for skill in support_topic_count[version].keys()])
    
    for skill in support_topic_count[version].keys():
        x0.append(skill)
        y0.append((support_topic_count[version][skill]["yes"] + support_topic_count[version][skill]["no"]) / total)
    
    fig.add_trace(go.Bar(
        x=x0,
        y=y0,
        name=version, 
        marker_color=colour_dict[version],
        opacity=0.75
    ))

fig.update_layout(
    title_text='Процент линков на скилл от всех линков в данной версии',
    xaxis_title_text='Skill',
    yaxis_title_text='Percent', 
    bargap=0.2,
    bargroupgap=0.1 
)

fig.show()